# REMOVE BLACKLIST REGIONS

# Prep deepTools multiBamSummary

## Purpose:

* Get Background Regions from FASTA File per Assay
* Parse FASTA Identifiers to Create BED Files
* Write BED Files

## Packages and Options

In [1]:
import glob, os
import pandas as pd


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Variables 

In [2]:
# lines vs CTRL whose background peaks we will pull from
lines=["56-22", "72-20"]

# list of ChIP-Seq Assays
chip_seq_assays=["H3K4me1","H3K4me3","H3K27Ac","H3K27me3"]

# dict for background regions per assay
assay_specific_background_regions={}

# dict to store BAM files for each assay for deepTools command 
assay_BAM_dict={}

In [3]:
# background FASTA directories for ATAC-Seq and ChIP-Seq
atac_seq_background_fasta_dir="/nfs/latdata/yraghav/CHDI/atac/reseqeuenced_data/homer/non-specific/background_peaks"
chip_seq_background_fasta_dir="/nfs/latdata/yraghav/CHDI/chip/high_coverage_chip/homer/background_peaks/fasta"

## Get Background Regions from FASTA File

### ATAC-Seq 

In [4]:
atac_seq_background_regions=[]

# for each line, glob background fasta file, append list with peak region line in fasta without ">"
for line in lines: 
    files = glob.glob("{}/{}*.fa".format(atac_seq_background_fasta_dir,line))
    assert len(files)==1, print(files)
    
    for file in files: 
        with open(file) as in_file: 
            for line in in_file: 
                if line.startswith(">"):
                    atac_seq_background_regions.append(line.strip("\n").split(">")[1])
                

In [5]:
assay_specific_background_regions["ATAC"]=atac_seq_background_regions

### ChIP-Seq

In [6]:

# for each ChIP-Seq Assay, for each line, glob background fasta file, append list with peak region line in fasta without ">"
for assay in chip_seq_assays: 
    
    chip_seq_background_regions=[]
    
    for line in lines: 
        files = glob.glob("{}/*{}*{}*.fa".format(chip_seq_background_fasta_dir,line, assay))
        assert len(files)==1, print(files)

        for file in files: 
            with open(file) as in_file: 
                for line in in_file: 
                    if line.startswith(">"):
                        chip_seq_background_regions.append(line.strip("\n").split(">")[1])
    
    assay_specific_background_regions[assay] = chip_seq_background_regions

## Sanity Check: Length of Regions Lists & Content

In [7]:
# check length of each object in dict 

for assay in assay_specific_background_regions:
    print("{}: {}".format(assay, len(assay_specific_background_regions[assay])))

ATAC: 393651
H3K4me1: 476714
H3K4me3: 138336
H3K27Ac: 170820
H3K27me3: 86865


In [8]:
for assay in assay_specific_background_regions:
    pd.Series(assay_specific_background_regions[assay])

0               chr1:66798-67060
1             chr1:180739-181148
2             chr1:629082-630053
3             chr1:630221-630479
4             chr1:630719-631404
                   ...          
393646    chrY:12446474-12446811
393647    chrY:12448053-12448296
393648    chrY:12450934-12451269
393649    chrY:24270746-24270988
393650    chrY:25399151-25399399
Length: 393651, dtype: object

0               chr1:777598-778094
1               chr1:812028-812445
2               chr1:843880-846421
3               chr1:858133-859349
4               chr1:904819-905892
                    ...           
476709    chrX:155765522-155767187
476710    chrX:155767962-155770339
476711    chrX:155770384-155771400
476712    chrX:155956864-155958131
476713    chrX:155959459-155960584
Length: 476714, dtype: object

0               chr1:777447-779497
1               chr1:779576-780466
2               chr1:825910-827899
3               chr1:828175-828433
4               chr1:844631-844974
                    ...           
138331    chrX:155957211-155958285
138332    chrX:155958666-155960245
138333    chrX:156016401-156016788
138334        chrY:5003106-5003427
138335      chrY:12564838-12565123
Length: 138336, dtype: object

0               chr1:778236-778644
1               chr1:778729-779409
2               chr1:826712-827771
3               chr1:958786-959213
4             chr1:1012976-1013414
                    ...           
170815    chrX:155641392-155641820
170816    chrX:155770104-155770431
170817    chrX:155881365-155881768
170818    chrX:155957874-155958216
170819    chrX:155958842-155959382
Length: 170820, dtype: object

0            chr1:788734-790364
1            chr1:860562-861171
2            chr1:861641-864435
3            chr1:865151-866637
4            chr1:866891-872775
                  ...          
86860    chrY:56870614-56871622
86861    chrY:56872038-56872550
86862    chrY:56872818-56873204
86863    chrY:56873557-56874690
86864    chrY:56879321-56879945
Length: 86865, dtype: object

## Convert Regions List to BED Format per Assay
* ## Write to BED File

In [9]:

# for each assay in dict of lists 
# load into pandas
# parse to get 3 separate columns (Chrom, Start, End) 
# write out to file 
for assay in assay_specific_background_regions: 
    
    tmp=pd.DataFrame(assay_specific_background_regions[assay])
    
    tmp["Chromosome"] = tmp[0].str.split(":").str[0]
    tmp["Start"] = tmp[0].str.split(":").str[1].str.split("-").str[0]
    tmp["End"] = tmp[0].str.split(":").str[1].str.split("-").str[1]

    tmp[["Chromosome","Start", "End"]].to_csv(
        "../output/raw_candidate_regions/{}.bed".format(assay), 
        sep="\t", index=False, header=False)
    
    

## Get Files

In [10]:
files = glob.glob("/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/*.bed")
files

['/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/ATAC.bed',
 '/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/H3K4me1.bed',
 '/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/H3K4me3.bed',
 '/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/H3K27Ac.bed',
 '/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/H3K27me3.bed']

## Create Bedtools Sort & Merge Commands 

In [11]:
bedtools_path="/nfs/apps/bedtools-2.26.0/bin/bedtools"

commands= []

for path in files: 
    
    commands.append(
        "{} sort -i {} | {} merge -i stdin | {} intersect -a stdin -b /home/yraghav/reference_inputs/bed/hg38.blacklist.bed -v > /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/{}".format(
            bedtools_path,
            path, 
            bedtools_path,
            bedtools_path,
            path.split("/")[-1].replace(".bed", ".sorted.merged.bed")  
            
        )
    ) 

## Print Commands (Copy & Paste to Terminal)

In [12]:
for command in commands: 
    print(command)

/nfs/apps/bedtools-2.26.0/bin/bedtools sort -i /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/ATAC.bed | /nfs/apps/bedtools-2.26.0/bin/bedtools merge -i stdin | /nfs/apps/bedtools-2.26.0/bin/bedtools intersect -a stdin -b /home/yraghav/reference_inputs/bed/hg38.blacklist.bed -v > /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/ATAC.sorted.merged.bed
/nfs/apps/bedtools-2.26.0/bin/bedtools sort -i /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/raw_candidate_regions/H3K4me1.bed | /nfs/apps/bedtools-2.26.0/bin/bedtools merge -i stdin | /nfs/apps/bedtools-2.26.0/bin/bedtools intersect -a stdin -b /home/yraghav/reference_inputs/bed/hg38.blacklist.bed -v > /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS

## Validate Output

In [13]:
print("\nFile and Number of Regions\n\n")

for file in glob.glob("/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/*.bed"): 
    num_lines = 0 
    
    with open(file,'r') as in_file: 
        for line in in_file: 
            num_lines+=1
    
    print("{}: {}".format(file, num_lines))


File and Number of Regions


/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/ATAC.sorted.merged.bed: 206607
/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/H3K4me1.sorted.merged.bed: 247764
/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/H3K4me3.sorted.merged.bed: 72216
/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/H3K27Ac.sorted.merged.bed: 93917
/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/H3K27me3.sorted.merged.bed: 45959


## Get BAM Files

### ATAC-Seq

In [14]:

assay_BAM_dict["ATAC"]=[]

# for each line
# glob for sample sheet corresponding to line
# read sample sheet in Pandas 
# subset "bamReads" column to list to extend "assay BAM dict"

for line in lines: 
    
    file = glob.glob("/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/atac/differential_analysis/sample_sheets/resequenced_data/*{}*.csv".format(line))
    assert len(file)==1, print(file)
    
    tmp_df = pd.read_csv(file[0], sep=",")
    assay_BAM_dict["ATAC"].extend(sorted(tmp_df["bamReads"].to_list()))
    
    

### ChIP-Seq

In [15]:

for assay in chip_seq_assays: 
    
    assay_BAM_dict[assay] = []
    
    for line in lines: 
    
        file = glob.glob("/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/chip/high_coverage/diff_analysis/sample_sheets/{}/*{}*.csv".format(
            assay, line
            )
        )
        
        assert len(file)==1, print(file)
        
        tmp_df = pd.read_csv(file[0], sep=",")
        assay_BAM_dict[assay].extend(sorted(tmp_df["bamReads"].to_list()))


### Make Unique List of BAM Files

In [16]:
# for each assay, unique list of BAM Files

for assay in assay_BAM_dict: 
    "Initial size: {}".format(len(assay_BAM_dict[assay]))
    
    assay_BAM_dict[assay] = list(set(assay_BAM_dict[assay]))
    
    "Final size: {}".format(len(assay_BAM_dict[assay]))


'Initial size: 18'

'Final size: 12'

'Initial size: 21'

'Final size: 14'

'Initial size: 18'

'Final size: 12'

'Initial size: 22'

'Final size: 14'

'Initial size: 20'

'Final size: 13'

## Make multiBamSummary Commands

In [18]:
counter = 1

with open("../deeptools_scripts/multiBamSummary.sh",'w') as out_file: 
    
    for assay in assay_BAM_dict: 
                
        out_file.write(
            "multiBamSummary BED-file --BED /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/sorted_merged_candidate_regions/{}.sorted.merged.bed --bamfiles {} -o /home/yraghav/DELETE_{}.npz --smartLabels --outRawCounts /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/matrices/raw/{}_raw_counts_matrix.tab --scalingFactors /home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/pre_processing/deeptools/output/scaling_factors/{}_scaling_factors.tab -p 60 \n".format(
                assay, 
                " ".join(assay_BAM_dict[assay]), 
                counter, 
                assay, 
                assay 
                
            )
        )
        
        counter+=1
    

2288

2636

2346

2636

2509